Background Information:
In 2011, a drought recorded as the worst in 60 years impacted East Africa. The food crisis that followed killed tens of thousands.

A long-standing conflict between two East African ethnic groups, the Turkana and the Pokot, intensified during the drought. Turkana cattle grazing lands in Kenya became desert-like, pushing cattle herds closer to Pokot territory. Pokot raiders attacked Turkana cattle herders in Kenya, then drove the cattle across the border to Uganda.

This conflict raises questions about the relationships between Kenya, Uganda, and the other countries of East Africa. How did Uganda and Kenya react to each other politically during the drought, and did other countries in the region react to the conflict?

TODO: Add a map of Turkana and Pokot territory. It should show the Turkana live in Keyna and the Pokot live in both Kenya and Uganda.



Hypothesis: 
Because a severe drought struck East Africa in 2011, conflicts directed toward Uganda and Kenya by other countries in the region increased during 2011. Both verbal and material conflicts became more common.

GDELT, a database of major world events, stores all of its data on world events with Google BigQuery. The SQL query below searches GDELT's BigQuery database for verbal and material conflict events directed toward Kenya in 2011. It also includes mapping information on the country which incited the event.

In [1]:
from google.cloud import bigquery

client = bigquery.Client()

In [2]:
sql = """
SELECT
    Year,
    Actor1Code,
    Actor2Code,
    QuadClass,
    Actor1Geo_Lat,
    Actor1Geo_Long
FROM
    `gdelt-bq.full.events`
WHERE
    Actor2Code = 'KEN'
    AND Year = 2011
    AND (QuadClass = 3 OR QuadClass = 4)

"""
df = client.query(sql).to_dataframe()

Show the first few rows of data.

In [3]:
df.head(9)

,Year,Actor1Code,Actor2Code,QuadClass,Actor1Geo_Lat,Actor1Geo_Long
0,2011,LBY,KEN,3,25.000000,17.0000
1,2011,ETH,KEN,4,3.500000,36.0000
2,2011,CRM,KEN,4,0.354615,37.5822
3,2011,GBRGOV,KEN,3,0.354615,37.5822
4,2011,CVL,KEN,3,-1.460950,37.4387
5,2011,LEG,KEN,3,-1.316670,36.7833
6,2011,JUD,KEN,4,-0.102210,34.7617
7,2011,GOV,KEN,3,0.178751,34.2954
8,2011,LEG,KEN,4,0.500000,36.0000


Use pandas to aggregate the data by location. Count the numbers of material and verbal events that started at the same latitude and longitude.

In [38]:
import pandas as pd

pivot = pd.pivot_table(df, index=['Actor1Geo_Lat', 'Actor1Geo_Long'], columns='QuadClass', aggfunc='size')
pivot = pivot.fillna(0)
pivot = pivot.reset_index()
pivot.head(9)

QuadClass,Actor1Geo_Lat,Actor1Geo_Long,3,4
0,-41.3000,174.7830,1.0,0.0
1,-41.0000,174.0000,0.0,2.0
2,-39.3333,174.2500,1.0,0.0
3,-39.0667,174.0830,2.0,0.0
4,-34.5875,-58.6725,1.0,0.0
5,-34.0000,-64.0000,2.0,0.0
6,-32.8000,-70.9167,1.0,0.0
7,-31.9333,115.8330,1.0,0.0
8,-30.1167,-52.0500,2.0,0.0


Make the data more usable for mapping. Sort the data to find which 100 locations started the most conflicts and label the conflict columns.

In [39]:
top_100_Verbal = pivot.sort_values(by=[3], ascending=False)[1:101]
top_100_Material = pivot.sort_values(by=[4], ascending=False)[1:101]

top_100_Verbal = top_100_Verbal.rename(columns={3: 'Verbal_Conflicts',
                                                4: 'Material_Conflicts'})
top_100_Material = top_100_Material.rename(columns={3: 'Verbal_Conflicts',
                                                4: 'Material_Conflicts'})


As an example of what the data looks like before any mapping, show some rows sorted by the number of verbal conflict events.

In [40]:
top_100_Verbal[:10]

QuadClass,Actor1Geo_Lat,Actor1Geo_Long,Verbal_Conflicts,Material_Conflicts
217,-1.283330,36.8167,1109.0,1201.0
558,1.000000,38.0000,736.0,676.0
718,10.000000,49.0000,268.0,653.0
591,2.066670,45.3667,252.0,596.0
876,38.000000,-97.0000,118.0,105.0
1016,54.000000,-2.0000,95.0,180.0
528,0.520356,35.2699,94.0,50.0
220,-1.283330,36.8667,75.0,31.0
401,-0.358173,42.5454,73.0,119.0
494,0.315556,32.5656,71.0,87.0
